In [48]:
import numpy as np
import networkx as nx
import heapq
from itertools import product
import os

data = "./mapcik.txt"
if not os.path.isfile(data):
    mapcik = [
    (1, 2),
    (1, 4),
    (2, 1),
    (2, 5),
    (3, 4),
    (3, 7),
    (4, 1),
    (4, 3),
    (4, 5),
    (4, 8),
    (5, 2),
    (5, 4),
    (5, 6),
    (5, 9),
    (6, 5),
    (6, 10),
    (7, 3),
    (7, 8),
    (8, 4),
    (8, 7),
    (8, 9),
    (8, 11),
    (9, 5),
    (9, 8),
    (9, 10),
    (9, 12),
    (10, 6),
    (10, 9),
    (11, 8),
    (11, 12),
    (12, 9),
    (12, 11),
],(1,3),(12,10)
    mapcik2 = [
    (10,11),
    (10,12),
    (11,30),
    (12,30),
    (20,21),
    (20,22),
    (21,30),
    (22,30),
    (30,15),
    (30,25)
],(10,20),(15,25)
    mapcik3 = [
        (1,2),
        (1,3),
        (2,1),
        (3,2)
    ], (1,2),(2,1)
    mapa,START,END = mapcik3
    net = nx.DiGraph()
    net.add_edges_from(mapa)
else:
    net = nx.read_edgelist("./mapcik.txt", edgetype=int, nodetype=int)

In [49]:

class Anode:
    __slots__ = ('state','f','g','parent','time')
    def __init__(self,state,f=None,g=0,parent=None,time=0):
        self.state=state
        self.f=f
        self.g = g
        self.parent=parent
        self.time=time
    def __lt__(self,o):
        return self.f<o.f
    def __repr__(self):
        return str(self.state)

def get_successor(state,adj):
    return product(*((u,)+tuple(adj[u]) for u in state))

def MAPF_valid(os,ns):
    return len(os) == len(set(ns)) and len(os) == len({frozenset(x) for x in zip(os,ns)})

def is_goal(state,goal):
    return state == goal

def get_path(node):
    vertexlist = [node.state]
    node = node.parent
    edgelist = []
    while node:
        edgelist.append(frozenset((node.state,vertexlist[-1])))
        vertexlist.append(node.state)
        node = node.parent
    vertexlist.reverse()
    edgelist.reverse()
    return vertexlist,edgelist

def reverse_dijkstra(graph, goals):
    rg = graph.reverse()
    h_values = []
    for goal in goals:
        h_values.append(nx.single_source_shortest_path_length(rg,goal))

    return h_values
def SIC(state,h_values):
    return sum([h_values[aidx][x] if x in h_values[aidx] else float("inf") for aidx,x in enumerate(state)])

def astar(G,start=(1,3),goal=(12,10),heuristic=None,heuristic_precalculator=None,constraint=None,agent=None):
    # Local variable cache
    heappush = heapq.heappush
    heappop = heapq.heappop
    adj = G.adj

    OPEN = []
    CLOSED = {}

    if heuristic_precalculator:
        h_values = heuristic_precalculator(G,goal)

    node = Anode(start,heuristic(start,h_values),0,None)
    OPENd = {(start,node.time):node.f}
    heappush(OPEN,node)

    while OPEN:
        q = heappop(OPEN)
        OPENd.pop((q.state,q.time))

        if is_goal(q.state,goal):return get_path(q)
        if (q.state,q.time) in CLOSED: continue
        CLOSED[(q.state,q.time)] = q.f
        for s in get_successor(q.state,adj):
            if not MAPF_valid(q.state,s): continue
            if (agent,s,q.time+1) in constraint: continue
            if (agent,frozenset((q.state,s)),q.time+1) in constraint: continue

            g = q.g+1
            h = heuristic(s,h_values)
            f = g+h

            n = Anode(s,f,g,q,q.time+1)
            if OPENd.get((n.state,n.time),float("inf")) > n.f:
                heappush(OPEN,n)
                OPENd[(n.state,n.time)] = n.f

In [50]:
class CTNode:
    def __init__(self,cost = None,con = None,vertexlist = None,edgelist = None):
        self.cost = cost
        self.con = con
        self.vertexlist = vertexlist
        self.edgelist = edgelist

    def root(self):
        self.cost = 0
        self.con = set()
        self.vertexlist = []
        self.edgelist = []

    def branch(self,new_con):
        return CTNode(self.cost,self.con.union(new_con),self.vertexlist.copy(),self.edgelist.copy())

    def __repr__(self):
        temp = []
        for i in self.vertexlist:
            temp.append(str([x[0] if len(x) == 1 else x for x in i]))
        return str(self.con) + "\nCost:" + str(self.cost) + "\nSolutions:\n\t" + "\n\t".join(temp)
    def __lt__(self,o):
        return self.cost<o.cost

def check_conflit(solution):
    conflit = False
    for timestep,i in enumerate(zip(*solution)):
        count={}
        for agent,j in enumerate(i):
            if count.get(j,0):
                vertex = j
                agent_j = agent
                agent_i = i.index(vertex)
                at = timestep
                conflit=True
                break
            count[j] = 1
        if conflit:
            break
    if conflit:
        return (agent_i,agent_j,vertex,at)
    return None

def high_level(G,start,goal):

    OPEN = []
    root = CTNode()
    root.root()
    for i in range(len(start)):
        x = astar(G,(start[i],),(goal[i],),heuristic=SIC,heuristic_precalculator=reverse_dijkstra,constraint=set(),agent=i)
        if x:
            root.vertexlist.append(x[0])
            root.edgelist.append(x[1])
            root.cost += len(root.vertexlist[-1])-1
        else:
            return

    heapq.heappush(OPEN,root)
    while OPEN:
        # input(":")
        # print()
        # print()
        # print()
        # print()
        # print()
        # for i in OPEN:
        #     print()
        #     print(i)
        q = heapq.heappop(OPEN)

        conflit = check_conflit(q.vertexlist)
        if not conflit:
            conflit = check_conflit(q.edgelist)
            if not conflit:
                return q.vertexlist

        for i in conflit[:2]:
            new = q.branch(set(((i,conflit[2],conflit[3]+1 if type(conflit[2]) == frozenset else conflit[3]),)))
            x = astar(G,(start[i],),(goal[i],),heuristic=SIC,heuristic_precalculator=reverse_dijkstra,constraint=new.con,agent=i)
            if x:
                new.vertexlist[i],new.edgelist[i] = x
                new.cost = q.cost+len(new.vertexlist[i])-len(q.vertexlist[i])
                heapq.heappush(OPEN,new)

raw = high_level(net,start=START,goal=END)
temp = []
for i in raw:
    temp.append(str([x[0] if len(x) == 1 else x for x in i]))
print("Final Solution:",end="")
print("",*temp,sep="\n\t")


Final Solution:
	[1, 3, 2]
	[2, 1]
